In [36]:
import torch
import spacy
import transformers
import itertools

In [2]:
tok = transformers.AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
model = transformers.AutoModel.from_pretrained('KB/bert-base-swedish-cased').eval()
nlp = spacy.load('../data/sv_model_xpos/sv_model0/sv_model0-0.0.0/')

In [10]:
for parameter in model.parameters():
    parameter.requires_grad=False

In [72]:
TXT="""Bob Dylan föddes som Robert Zimmerman i staden Duluth, Minnesota men strax innan han fyllde sex år och efter att hans far fått polio flyttade familjen till den närliggande staden Hibbing, Minnesota där han sedan växte upp. Familjen Zimmerman var judisk och deras förfäder hade utvandrat från Ryssland, Ukraina, Litauen och Turkiet. Morfar och mormor - Benjamin och Liba Edelstein (senare Stein och Stone) - var litauiska judar som emigrerade till USA 1902.
När Bob Dylan var åtta-nio år började han spela på familjens piano. Därefter lärde han sig att spela munspel och gitarr.[3] Mycket av hans ungdomstid gick åt till att lyssna på radio där han tog in stationer som sände blues, country och tidig rock'n'roll. Han började uppträda i mitten av 1950-talet och var medlem i ett flertal band under sin tid i high school.

1959 började han studera på universitetet i Minneapolis. I samma veva tog hans intresse för folkmusik fart. Det var också nu han började presentera sig som Bob Dylan. Var han fått namnet ifrån finns det flera historier om. Vissa menar att det är inspirerat av poeten Dylan Thomas. År 2004 skrev han själv om hur han valde namnet i sin bok Memoarer, första delen"""



In [291]:
def parse_sentence(sentence):
    doc = nlp(sentence)
    #length = 1
    #input_ids = [tok.cls_token_id]
    #word_ends = [1]
    
    length = 0
    input_ids = []
    word_start = []
    
    
    
    ixs, tokens = zip(*[(ix, token) for (ix, token) in enumerate(doc) if not token.is_space])
    
    # [[0], [1,2,3], [3,2]]
    wordpieces_for_token = tok(
            [token.string for token in tokens],
            add_special_tokens=False, 
            padding=False, 
            return_token_type_ids=False, 
            return_attention_mask=False)['input_ids']
    
        
    
    for wordpieces in wordpieces_for_token:
        word_start.append(length)
        input_ids += wordpieces
        assert len(wordpieces) > 0, "Empty token makes program sad"
        length += len(wordpieces)
    
    nouns = []
    spans = []
    ptr = 0
    
    chunkfix = {i:j for j,i in enumerate(ixs)}
    for chunk in doc.noun_chunks:
        start = chunkfix[chunk.start]
        end = chunkfix[chunk.end]
        #Add the next chunk to spans        
        for i in range(ptr, start):
            #Add all non noun chunks to the span
            nouns.append(False)
            spans.append(word_start[i])
            
        #Add the chunk to the spans
        nouns.append(True)
        spans.append(word_start[start])
        
        ptr = end
        
    #print("adding all words after last noun chunk at: ", i
    nouns.extend([False for start in word_start[ptr:]])
    spans.extend([start for start in word_start[ptr:]])
    
    input_ids = torch.LongTensor([tok.cls_token_id, *input_ids])
    nouns = [False, *nouns]
    spans = [0, *[start + 1 for start in spans]]
    
    spans = list(zip(spans, spans[1:] + [len(input_ids)]))
    #spans = [(False, 0), *[(noun,start+1) for noun, start in spans]]
    
    return input_ids, nouns, spans

input_ids, nouns, spans = parse_sentence(TXT)


In [272]:
nouns

[False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 Fa

In [274]:
test = []
for noun, (start, stop) in zip(nouns, spans):
    if noun: 
        test.append(tok.decode(input_ids[start:stop]))
        
for a, b in zip(test, doc.noun_chunks):
    print('{}\n{}\n\n'.format(a,b))

Bob Dylan
Bob Dylan


i staden
i staden


han
han


sex år
sex år


hans far
hans far


polio
polio


familjen
familjen


den närliggande staden
den närliggande staden


, Minnesota
, Minnesota


han
han


Familjen Zimmerman
Familjen Zimmerman


och deras förfäder
och deras förfäder


som
som


Bob Dylan
Bob Dylan


han
han


familjens
familjens


han
han


sig
sig


munspel och gitarr. [ 3
munspel och gitarr.[3


] Mycket av hans ungdomstid
] Mycket av hans ungdomstid


han
han


stationer som sände blues, country och tidig rock'n'roll
stationer som sände blues, country och tidig rock'n'roll


Han
Han


av 1950 - talet
av 1950-talet


ett flertal
ett flertal


under sin tid
under sin tid


han
han


i Minneapolis
i Minneapolis


hans intresse för folkmusik fart
hans intresse för folkmusik fart


han
han


sig
sig


han
han


namnet
namnet


det
det


flera historier om
flera historier om


Vissa
Vissa


det
det


poeten
poeten


År
År


han själv
han själv


han
han


namnet
namnet




In [275]:
def get_attention(input_ids, spans):
    # Average attention over heads in the last layer
    att = model(input_ids.unsqueeze(0), output_attentions=True)['attentions'][-1].mean(1).squeeze(0)

    word_span = spans #list(zip(word_start, word_end))
    
    N = len(word_start)
    
    tmp = att.new_zeros((N,N))

    tmp[0,0] = att[0,0]
    
    for i in range(1,N):
        istart, iend = word_span[i]
        tmp[0, i] = att[0, istart:iend].sum()
        tmp[i, 0] = att[istart:iend, 0].sum() / (iend-istart)
    
    for i,j in itertools.product(range(1,N),range(1,N)):
        istart, iend = word_span[i]
        jstart, jend = word_span[j]
        tmp[i,j] = att[istart:iend, jstart:jend].sum() / (iend-istart)
    
    return tmp

att = get_attention(input_ids, spans)

In [278]:
def print_wix(ix):
    start, stop = spans[ix]
    return tok.decode(input_ids[start:stop])

In [290]:
noun_ixs = [ix for ix, noun in enumerate(nouns) if noun]
rel_ixs = [ix for ix, noun in enumerate(nouns) if not noun]
rixs = torch.LongTensor(rel_ixs)



for head, tail in itertools.product(noun_ixs, noun_ixs):
    if abs(head - tail) <= 1:
        continue
        
    rixs = torch.LongTensor([ix for ix, noun in enumerate(nouns) if (not noun) and (head < ix < tail or head > ix > tail)])
    scores = att[head, rixs] + att[rixs, tail]
    best = torch.argmax(scores)
    rel = rixs[best].item()
    score = scores[best].item()
    print('{:.3f}: {} -- {} -- {}'.format(score, print_wix(head), print_wix(rel), print_wix(tail)))

0.016: Bob Dylan -- Robert -- i staden
0.020: Bob Dylan -- innan -- han
0.035: Bob Dylan -- fyllde -- sex år
0.036: Bob Dylan -- att -- hans far
0.036: Bob Dylan -- fått -- polio
0.013: Bob Dylan -- Zimmerman -- familjen
0.021: Bob Dylan -- till -- den närliggande staden
0.058: Bob Dylan -- , -- , Minnesota
0.012: Bob Dylan -- Zimmerman -- han
0.138: Bob Dylan -- . -- Familjen Zimmerman
0.136: Bob Dylan -- . -- och deras förfäder
0.128: Bob Dylan -- . -- som
0.141: Bob Dylan -- . -- Bob Dylan
0.133: Bob Dylan -- . -- han
0.133: Bob Dylan -- . -- familjens
0.133: Bob Dylan -- . -- han
0.134: Bob Dylan -- . -- sig
0.165: Bob Dylan -- . -- munspel och gitarr. [ 3
0.159: Bob Dylan -- . -- ] Mycket av hans ungdomstid
0.134: Bob Dylan -- . -- han
0.175: Bob Dylan -- . -- stationer som sände blues, country och tidig rock'n'roll
0.134: Bob Dylan -- . -- Han
0.142: Bob Dylan -- . -- av 1950 - talet
0.137: Bob Dylan -- . -- ett flertal
0.140: Bob Dylan -- . -- under sin tid
0.133: Bob Dylan -- .

0.155: sig -- . -- , Minnesota
0.150: sig -- . -- han
0.160: sig -- . -- Familjen Zimmerman
0.158: sig -- . -- och deras förfäder
0.150: sig -- . -- som
0.156: sig -- . -- Bob Dylan
0.149: sig -- . -- han
0.149: sig -- . -- familjens
0.165: sig -- spela -- munspel och gitarr. [ 3
0.008: sig -- att -- ] Mycket av hans ungdomstid
0.007: sig -- där -- han
0.065: sig -- spela -- stationer som sände blues, country och tidig rock'n'roll
0.143: sig -- . -- Han
0.151: sig -- . -- av 1950 - talet
0.146: sig -- . -- ett flertal
0.149: sig -- . -- under sin tid
0.142: sig -- . -- han
0.150: sig -- . -- i Minneapolis
0.155: sig -- . -- hans intresse för folkmusik fart
0.142: sig -- . -- han
0.142: sig -- . -- sig
0.142: sig -- . -- han
0.143: sig -- . -- namnet
0.143: sig -- . -- det
0.154: sig -- . -- flera historier om
0.144: sig -- . -- Vissa
0.142: sig -- . -- det
0.142: sig -- . -- poeten
0.144: sig -- . -- År
0.147: sig -- . -- han själv
0.142: sig -- . -- han
0.143: sig -- . -- namnet
0.145

0.047: namnet -- i -- under sin tid
0.017: namnet -- fått -- han
0.017: namnet -- fått -- i Minneapolis
0.035: namnet -- nu -- hans intresse för folkmusik fart
0.033: namnet -- presentera -- han
0.021: namnet -- fått -- sig
0.032: namnet -- fått -- han
0.027: namnet -- ifrån -- det
0.052: namnet -- finns -- flera historier om
0.028: namnet -- ifrån -- Vissa
0.037: namnet -- ifrån -- det
0.029: namnet -- ifrån -- poeten
0.028: namnet -- ifrån -- År
0.032: namnet -- ifrån -- han själv
0.026: namnet -- ifrån -- han
0.037: namnet -- valde -- namnet
0.029: namnet -- ifrån -- sin bok
0.147: det -- . -- Bob Dylan
0.143: det -- . -- i staden
0.140: det -- . -- han
0.144: det -- . -- sex år
0.143: det -- . -- hans far
0.145: det -- . -- polio
0.140: det -- . -- familjen
0.147: det -- . -- den närliggande staden
0.146: det -- . -- , Minnesota
0.141: det -- . -- han
0.150: det -- . -- Familjen Zimmerman
0.149: det -- . -- och deras förfäder
0.140: det -- . -- som
0.146: det -- . -- Bob Dylan
0.13

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183]

In [256]:
black_list_relation = set([ token2id[n]  for n in noun_chunks ])
all_relation_pairs = []
id2token = { value: key for key, value in token2id.items()}
with Pool(10) as pool:
    params = [  ( pair[0], pair[1], attn_graph, max(tokenid2word_mapping), black_list_relation, ) for pair in tail_head_pairs]
    for output in pool.imap_unordered(bfs, params):
    if len(output):
        all_relation_pairs += [ (o, id2token) for o in output ]
        
triplet_text = []
with Pool(10, global_initializer, (nlp,)) as pool:
    for triplet in pool.imap_unordered(filter_relation_sets, all_relation_pairs):
        if len(triplet) > 0:
            triplet_text.append(triplet)
return triplet_text

AttributeError: 'builtin_function_or_method' object has no attribute 'triu_like'

In [255]:
att.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 

In [230]:
def get_attention(input_ids, doc, word_ends):
    # Average attention over heads in the last layer
    att = model(input_ids.unsqueeze(0), output_attentions=True)['attentions'][-1].mean(1).squeeze(0)
    
    N = len(word_ends)
    
    tmp = att.new_zeros((N,N))

    tmp[0,0] = att[0,0]
    
    for i in range(1,N):
        istart, iend = word_ends[i-1], word_ends[i]
        tmp[0, i] = att[0, istart:iend].sum()
        tmp[i, 0] = att[istart:iend, 0].sum() / (iend-istart)
    
    for i,j in itertools.product(range(1,N),range(1,N)):
        istart, iend = word_ends[i-1], word_ends[i]
        jstart, jend = word_ends[j-1], word_ends[j]
        tmp[i,j] = att[istart:iend, jstart:jend].sum() / (iend-istart)
        
#    cs = att.cumsum(0).cumsum(1)
#    tmp2 = att.new_zeros((N,N))
#
#    tmp2[0,0] = att[0,0]
#    
#    for i in range(1, N):
#        istart, iend = word_ends[i-1]-1, word_ends[i]-1
#        jstart, jend = 0, 1
#        tmp2[0,i] = cs[jend, iend] - cs[jstart, iend] - cs[jend, istart] + cs[jstart,istart]     
#        tmp2[i,0] = cs[iend, jend] - cs[istart, jend] - cs[iend, jstart] + cs[istart,jstart]
#        tmp2[i,0] /= iend - istart
#    
#    for i,j in itertools.product(range(1,N),range(1,N)):
#        istart, iend = word_ends[i-1]-1, word_ends[i]-1
#        jstart, jend = word_ends[j-1]-1, word_ends[j]-1
#        tmp2[i,j] = cs[iend, jend] - cs[istart, jend] - cs[iend, jstart] + cs[istart,jstart]
#        tmp2[i,j] /= iend - istart
    
    return tmp

att = get_attention(input_ids, doc, word_ends)

In [231]:
att

tensor([[0.0238, 0.0238, 0.0038,  ..., 0.0007, 0.0005, 0.0003],
        [0.0238, 0.0238, 0.0038,  ..., 0.0007, 0.0005, 0.0003],
        [0.0029, 0.0029, 0.1028,  ..., 0.0003, 0.0005, 0.0002],
        ...,
        [0.0006, 0.0006, 0.0026,  ..., 0.0319, 0.0034, 0.0029],
        [0.0007, 0.0007, 0.0016,  ..., 0.0126, 0.0661, 0.0146],
        [0.0003, 0.0003, 0.0029,  ..., 0.0111, 0.0545, 0.1788]])

In [232]:
att.sum(1)

tensor([1.0238, 1.0238, 1.0029, 1.0003, 1.0004, 1.0015, 1.0018, 1.0002, 1.0003,
        1.0002, 1.0005, 1.0002, 1.0001, 1.0001, 1.0006, 1.0001, 1.0001, 1.0003,
        1.0003, 1.0003, 1.0009, 1.0002, 1.0003, 1.0005, 1.0014, 1.0007, 1.0010,
        1.0008, 1.0017, 1.0014, 1.0035, 1.0018, 1.0008, 1.0013, 1.0032, 1.0068,
        1.0032, 1.0065, 1.0061, 1.0050, 1.0024, 1.0023, 1.0033, 1.0020, 1.0019,
        1.0019, 1.0018, 1.0026, 1.0028, 1.0049, 1.0041, 1.0033, 1.0047, 1.0016,
        1.0035, 1.0021, 1.0016, 1.0024, 1.0015, 1.0013, 1.0033, 1.0028, 1.0039,
        1.0015, 1.0021, 1.0009, 1.0016, 1.0025, 1.0011, 1.0005, 1.0011, 1.0015,
        1.0011, 1.0032, 1.0019, 1.0050, 1.0013, 1.0023, 1.0029, 1.0021, 1.0088,
        1.0019, 1.0030, 1.0039, 1.0032, 1.0032, 1.0075, 1.0052, 1.0128, 1.0069,
        1.0088, 1.0041, 1.0071, 1.0187, 1.0053, 1.0090, 1.0107, 1.0120, 1.0180,
        1.0125, 1.0392, 1.0102, 1.0144, 1.0111, 1.0113, 1.0018, 1.0033, 1.0032,
        1.0010, 1.0023, 1.0006, 1.0009, 

In [26]:
csatt = att.cumsum(0).cumsum(1)

In [31]:
i = 23
j = 65
csatt[0, j] - csatt[0, i] 

tensor(0.1238)

In [32]:
att[0, i:j+1].sum()

tensor(0.1241)

In [ ]:
with torch.no_grad():
    if use_cuda:
        for key in inputs.keys():
        inputs[key] = inputs[key].cuda()
        outputs = encoder(**inputs, output_attentions=True)
trim = True
'''
Use average of last layer attention : page 6, section 3.1.2
'''
attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)
merged_attention = compress_attention(attention, tokenid2word_mapping)
attn_graph = build_graph(merged_attention)

In [ ]:
for (x, start, end) in chunk_spans:
    print(x, tok.decode(input_ids[start:end]))

In [ ]:
chunk.start
chunk.end

In [ ]:
def parse_sentence(sentence, tok_params):
    huggingface_encoding = tok(sentence, **tok_params)
    doc = nlp(sentence)
    tokens = list(doc)
    chunk2id = {}
    start_chunk = []
    end_chunk = []
    noun_chunks = []
    for chunk in doc.noun_chunks:
        noun_chunks.append(chunk.text)
        start_chunk.append(chunk.start)
        end_chunk.append(chunk.end)

In [ ]:
class nlp:
    def __init__(self, sentence):de
        self.text = sentence
        self.noun_chunks = [
        a
def parse_sentence(*args, **kwargs):
    """ parse sentence """
    inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)
    triplets = []
    return triplets

def create_mapping(sentence, return_pt, nlp, tokenizer):
    inputs = None
    tokenid2word = None
    token2id = None
    noun_chunks = None
    return inputs, tokenid2word_mapping, token2id, noun_chunks

tokenizer = None
encoder = None

sentence = 'Bob Dylan är en gubbe som har skrivit musik.'

triplets = parse_sentence(sent.text, tokenizer, encoder, nlp)

In [ ]:
def create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer):
    return None

def parse_sentence(sentence, tokenizer, encoder, nlp, use_cuda=True):
    ''' Parse stuff '''
    tokenizer_name = str(tokenizer.__str__)

    inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)

    with torch.no_grad():
        if use_cuda:
            for key in inputs.keys():
                inputs[key] = inputs[key].cuda()
        outputs = encoder(**inputs, output_attentions=True)
    trim = True
    if 'GPT2' in tokenizer_name:
        trim  = False

    ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff ''' Parse stuff '''
    Use average of last layer attention : page 6, section 3.1.2
    '''
    attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)

    merged_attention = compress_attention(attention, tokenid2word_mapping)
    attn_graph = build_graph(merged_attention)

    tail_head_pairs = []
    for head in noun_chunks:
        for tail in noun_chunks:
            if head != tail:
                tail_head_pairs.append((token2id[head], token2id[tail]))

    black_list_relation = set([ token2id[n]  for n in noun_chunks ])

    all_relation_pairs = []
    id2token = { value: key for key, value in token2id.items()}

    with Pool(10) as pool:
        params = [  ( pair[0], pair[1], attn_graph, max(tokenid2word_mapping), black_list_relation, ) for pair in tail_head_pairs]
        for output in pool.imap_unordered(bfs, params):
            if len(output):
                all_relation_pairs += [ (o, id2token) for o in output ]

    triplet_text = []
    with Pool(10, global_initializer, (nlp,)) as pool:
        for triplet in pool.imap_unordered(filter_relation_sets, all_relation_pairs):
            if len(triplet) > 0:
                triplet_text.append(triplet)
    return triplet_text

In [ ]:
nlp = spacy.load('xx_ent_wiki_sm')
#nlp = spacy.load('da_core_news_md')

In [ ]:
x = nlp(sentence)

In [ ]:
list(x.noun_chunks)